In [1]:
import pandas as pd

In [2]:
# сколько нужно денег выделить на рекламу в следующем месяце, 
# чтобы увеличить выручку в 2 раза

# сейчас выручка = X = 21K 
# нам в следующем месяце нужно получить 2X = 42K
# итоговая выручка = 
    # выручка от новых клиентов = $27.3K
    # + выручка от текущих клиентов = 14.7K
# какую долю выручки можно ожидать от текущих клиентов?
# остаток — то, что нужно «докупить» маркетингом

In [3]:
# 1. Построим календарный когортный отчет
# 2. Считаем долю выручки от новых клиентов
# ...

In [ ]:
# Алгоритм
# 0. Формулируем задачу
# 1. Определяем когорту (событие и временной промежуток) — первая покупка по месяцам
# 2. Выделяем целевую метрику - выручка
# 3. Выбираем подходящий формат отчета - календарный

In [27]:
orders = pd.read_csv('data/orders.csv', index_col='payment_id')
orders['created_at'] = pd.to_datetime(orders['created_at'])
orders = (
    orders
    .groupby('user_id')
    .agg(first_payment_at = ('created_at','min'))
    .merge(orders, how='inner', left_index=True, right_on='user_id')
)
orders['payment_month'] = orders['created_at'].dt.to_period('M')
orders['first_payment_month'] = orders['first_payment_at'].dt.to_period('M')

In [33]:
(
    orders
    .groupby(['first_payment_month', 'payment_month'])
    .agg(revenue = ('payment_sum', 'sum'))
).tail(5)

revenue
first_payment_month payment_month         
2022-09             2022-10        2385.74
                    2022-11        1534.92
2022-10             2022-10        6904.14
                    2022-11        1846.74
2022-11             2022-11        5550.26

In [43]:
cohorts = (
    orders
    .pivot_table(
        index='first_payment_month',
        columns='payment_month',
        values='payment_sum',
        aggfunc='sum'
    )
)

In [69]:
cohorts.sum(axis='rows')

payment_month
2019-12      479.20
2020-01     3105.00
2020-02     3959.21
2020-03     3830.15
2020-04     5178.73
2020-05     6075.07
2020-06     7539.99
2020-07    10398.12
2020-08    10707.28
2020-09     9926.86
2020-10    11331.56
2020-11    11505.93
2020-12    10275.28
2021-01    11432.94
2021-02     8725.90
2021-03     9855.15
2021-04    11780.69
2021-05    11519.70
2021-06    13674.74
2021-07    12210.66
2021-08    12443.40
2021-09    10543.05
2021-10    12531.69
2021-11    14753.47
2021-12    13861.34
2022-01    14207.26
2022-02    15417.54
2022-03    17315.27
2022-04    15995.23
2022-05    19872.96
2022-06    21304.66
2022-07    30134.08
2022-08    26030.25
2022-09    24053.61
2022-10    24405.30
2022-11    21906.35
Freq: M, dtype: float64

In [68]:
[cohorts.at[m,m] for m in cohorts.columns] / cohorts.sum(axis='rows')

payment_month
2019-12    1.000000
2020-01    0.967250
2020-02    0.826640
2020-03    0.782758
2020-04    0.528778
2020-05    0.568927
2020-06    0.489693
2020-07    0.515731
2020-08    0.464140
2020-09    0.414599
2020-10    0.391675
2020-11    0.342659
2020-12    0.336693
2021-01    0.428515
2021-02    0.257788
2021-03    0.299492
2021-04    0.345021
2021-05    0.381927
2021-06    0.379585
2021-07    0.399384
2021-08    0.322369
2021-09    0.321196
2021-10    0.356536
2021-11    0.428804
2021-12    0.316025
2022-01    0.346994
2022-02    0.310629
2022-03    0.325305
2022-04    0.323955
2022-05    0.383911
2022-06    0.329473
2022-07    0.313939
2022-08    0.287265
2022-09    0.281811
2022-10    0.282895
2022-11    0.253363
Freq: M, dtype: float64